**각 Top Features만을 사용한 RandomForestClassifier와 LSTM의 성능 비교**

In [78]:
import numpy as np, os
import pandas as pd

import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import warnings
warnings.filterwarnings(action='ignore')

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import Dropout, InputLayer
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

print(tf.__version__)
print(keras.__version__)

#한글설정
import matplotlib.font_manager as fm

font_dirs = ['/usr/share/fonts/truetype/nanum', ]
font_files = fm.findSystemFonts(fontpaths=font_dirs)

for font_file in font_files:
    fm.fontManager.addfont(font_file)
    
# 한글 출력을 위해서 폰트 옵션을 설정합니다.
# "axes.unicode_minus" : 마이너스가 깨질 것을 방지

sns.set(font="NanumBarunGothic", 
        rc={"axes.unicode_minus":False},
        style='darkgrid')

2.5.0
2.5.0


# RandomForest FI Top10 list
1. Feature Importance를 구하여 Top Features 추출
2. Top Features만을 사용한 데이터로 RandomForestClassifier 학습 후 정확도 측정

In [2]:
COLS = list(pd.read_csv('/project/LSH/total_data_7727.csv')['ITEMID'].sort_values().unique())

In [46]:
import random
random.seed(42)
path = '/project/LSH/'
x =  np.load(path + 'x_(7727,4068).npy')
y = np.load(path + 'y_(7727,1).npy')

In [47]:
x.shape

(7727, 4068)

In [48]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics 
np.random.seed(42)
model = RandomForestClassifier()
model.fit(x, y)

pred = model.predict(x)

print(f'정확도 : {metrics.accuracy_score(y, pred)}')

정확도 : 1.0


In [49]:
importances = model.feature_importances_
importances

array([1.40546352e-03, 8.98727598e-04, 3.07665371e-03, ...,
       3.64771738e-04, 9.96495876e-05, 4.21937811e-05])

In [50]:
result = []
for f in range(x.shape[1]):
    result.append({'feature' : COLS[f], 'importances' : importances[f]})

In [51]:
rf_fi = pd.DataFrame(result).sort_values('importances', ascending=False)
rf_fi

,feature,importances
206,51277,0.013739
121,51006,0.009766
56,50912,0.009121
103,50983,0.007452
20,50862,0.007445
...,...,...
2281,472120006,0.000000
1604,131541070,0.000000
3217,51079079420,0.000000
1042,62401002,0.000000


In [139]:
rf_feature = rf_fi.feature[:30].to_list()
rf_feature

[51277,
 51006,
 50912,
 50983,
 50862,
 63323026201,
 51003,
 904224461,
 50882,
 50868,
 51275,
 50813,
 227194,
 51244,
 409606211,
 51274,
 50971,
 245004101,
 51256,
 51079000220,
 51493,
 55390000401,
 50960,
 50902,
 51265,
 51248,
 224275,
 51249,
 50893,
 50970]

In [140]:
#COLS에서의 index 추출 (COLS의 index가 곧 3차원데이터 feature의 index이기 때문)

rf_feature_index = [COLS.index(i) for i in rf_feature]
rf_feature_index

[206,
 121,
 56,
 103,
 20,
 3832,
 119,
 2611,
 34,
 26,
 204,
 8,
 383,
 184,
 2195,
 203,
 95,
 1903,
 193,
 3045,
 278,
 3456,
 86,
 48,
 197,
 186,
 305,
 187,
 41,
 94]

In [141]:
#검증
for i in rf_feature_index:
    print(COLS[i])

51277
51006
50912
50983
50862
63323026201
51003
904224461
50882
50868
51275
50813
227194
51244
409606211
51274
50971
245004101
51256
51079000220
51493
55390000401
50960
50902
51265
51248
224275
51249
50893
50970


# LSTM FI Top10 list

In [161]:
lstm_fi = pd.read_csv('./data/method1_score.csv')
lstm_fi['abs_score'] = lstm_fi['score'].apply(lambda x:abs(x))
lstm_fi = lstm_fi.sort_values('abs_score', ascending=False)
lstm_feature = lstm_fi.feature[:30].to_list()
lstm_feature

[409606211,
 74606211,
 67434504,
 54817525,
 10019055302,
 227194,
 54858516,
 74706811,
 45006701,
 4003822,
 228125,
 50889,
 63323016501,
 58177025504,
 182138167,
 51479005520,
 51079007520,
 51200,
 406112101,
 25021011210,
 63323016101,
 58468002101,
 781155613,
 224270,
 245021201,
 51079008620,
 50383068304,
 172572810,
 63323030201,
 63323038810]

In [162]:
#COLS에서의 index 추출 (COLS의 index가 곧 3차원데이터 feature의 index이기 때문)

lstm_feature_index = [COLS.index(i) for i in lstm_feature]
lstm_feature_index

[2195,
 1276,
 1081,
 982,
 2744,
 383,
 1011,
 1306,
 845,
 469,
 392,
 37,
 3820,
 3557,
 1815,
 3283,
 3070,
 162,
 2110,
 2860,
 3818,
 3567,
 2547,
 301,
 1915,
 3076,
 2988,
 1722,
 3841,
 3852]

# With RF FI Top10

## RF

In [144]:
path = '/project/LSH/'
X =  np.load(path + 'x_(7727,4068).npy')[:,rf_feature_index]
y = np.load(path + 'y_(7727,1).npy')

idx = list(range(len(X)))
random.shuffle(idx)

i = round(X.shape[0]*0.8)
X_train, y_train = X[idx[:i],:], y[idx[:i]]
X_test, y_test = X[idx[i:],:], y[idx[i:]]

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((6182, 30), (6182,), (1545, 30), (1545,))

In [145]:
model = RandomForestClassifier()
model.fit(X_train, y_train)

pred = model.predict(X_test)

print(f'정확도 : {metrics.accuracy_score(y_test, pred)}')

정확도 : 0.7242718446601941


## LSTM

In [146]:
import random    
seed_num = 42
random.seed(seed_num)
path = '/project/LSH/'
X = np.load(path + 'x_(7727,10,4068).npy')[:,:,rf_feature_index]
y = np.load(path + 'y_(7727,1).npy')

idx = list(range(len(X)))
random.shuffle(idx)


i = round(X.shape[0]*0.8)
X_train, y_train = X[idx[:i],:,:], y[idx[:i]]
X_test, y_test = X[idx[i:],:,:], y[idx[i:]]

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((6182, 10, 30), (6182,), (1545, 10, 30), (1545,))

In [147]:
# ---------------------
seed_num = 42 
# ---------------------
tf.random.set_seed(seed_num)

lstm = Sequential()
lstm.add(InputLayer(input_shape=(X_train.shape[1],X_train.shape[2])))
lstm.add(LSTM(units=128, activation='hard_sigmoid', return_sequences=True))
lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
lstm.add(Dropout(0.2))
lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
lstm.add(LSTM(units=32, activation='hard_sigmoid', return_sequences=False))
lstm.add(Dropout(0.2))
lstm.add(Dense(units=1, activation='sigmoid'))

from tensorflow.keras.callbacks import ModelCheckpoint
import os

MODEL_SAVE_FOLDER_PATH = './model/'
if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
    os.mkdir(MODEL_SAVE_FOLDER_PATH)

model_path = MODEL_SAVE_FOLDER_PATH + 'rf_top10_seed42-{epoch:02d}-{val_loss:.4f}.hdf5'

cb_checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss',
                                verbose=1, save_best_only=True)

early_stop = EarlyStopping(monitor='val_acc', patience=50, verbose=1, restore_best_weights=True)
lstm.compile(optimizer= keras.optimizers.Adam(learning_rate = 0.001), loss = "binary_crossentropy", metrics=['acc'])
lstm.fit(X_train, y_train, validation_split=0.25, batch_size=128, epochs=500,  callbacks=[early_stop,cb_checkpoint], shuffle=False)

Epoch 1/500
37/37 [==============================] - 5s 54ms/step - loss: 0.6758 - acc: 0.5984 - val_loss: 0.6659 - val_acc: 0.6177

Epoch 00001: val_loss improved from inf to 0.66589, saving model to ./model/rf_top10_seed42-01-0.6659.hdf5
Epoch 2/500
37/37 [==============================] - 2s 45ms/step - loss: 0.6728 - acc: 0.6070 - val_loss: 0.6655 - val_acc: 0.6177

Epoch 00002: val_loss improved from 0.66589 to 0.66547, saving model to ./model/rf_top10_seed42-02-0.6655.hdf5
Epoch 3/500
37/37 [==============================] - 2s 44ms/step - loss: 0.6680 - acc: 0.6091 - val_loss: 0.6442 - val_acc: 0.6177

Epoch 00003: val_loss improved from 0.66547 to 0.64418, saving model to ./model/rf_top10_seed42-03-0.6442.hdf5
Epoch 4/500
37/37 [==============================] - 2s 44ms/step - loss: 0.6175 - acc: 0.6450 - val_loss: 0.5856 - val_acc: 0.6986

Epoch 00004: val_loss improved from 0.64418 to 0.58557, saving model to ./model/rf_top10_seed42-04-0.5856.hdf5
Epoch 5/500
37/37 [=========

37/37 [==============================] - 2s 44ms/step - loss: 0.5125 - acc: 0.7489 - val_loss: 0.5250 - val_acc: 0.7219

Epoch 00036: val_loss did not improve from 0.52433
Epoch 37/500
37/37 [==============================] - 2s 46ms/step - loss: 0.5078 - acc: 0.7496 - val_loss: 0.5258 - val_acc: 0.7251

Epoch 00037: val_loss did not improve from 0.52433
Epoch 38/500
37/37 [==============================] - 2s 44ms/step - loss: 0.5096 - acc: 0.7481 - val_loss: 0.5263 - val_acc: 0.7277

Epoch 00038: val_loss did not improve from 0.52433
Epoch 39/500
37/37 [==============================] - 2s 45ms/step - loss: 0.5086 - acc: 0.7511 - val_loss: 0.5251 - val_acc: 0.7245

Epoch 00039: val_loss did not improve from 0.52433
Epoch 40/500
37/37 [==============================] - 2s 44ms/step - loss: 0.5067 - acc: 0.7457 - val_loss: 0.5247 - val_acc: 0.7245

Epoch 00040: val_loss did not improve from 0.52433
Epoch 41/500
37/37 [==============================] - 2s 45ms/step - loss: 0.5081 - acc:

37/37 [==============================] - 2s 45ms/step - loss: 0.4703 - acc: 0.7705 - val_loss: 0.5399 - val_acc: 0.7206

Epoch 00080: val_loss did not improve from 0.52342
Epoch 81/500
37/37 [==============================] - 2s 44ms/step - loss: 0.4710 - acc: 0.7649 - val_loss: 0.5384 - val_acc: 0.7212

Epoch 00081: val_loss did not improve from 0.52342
Epoch 82/500
37/37 [==============================] - 2s 45ms/step - loss: 0.4675 - acc: 0.7642 - val_loss: 0.5401 - val_acc: 0.7212

Epoch 00082: val_loss did not improve from 0.52342
Epoch 83/500
37/37 [==============================] - 2s 44ms/step - loss: 0.4634 - acc: 0.7692 - val_loss: 0.5436 - val_acc: 0.7206

Epoch 00083: val_loss did not improve from 0.52342
Epoch 84/500
37/37 [==============================] - 2s 44ms/step - loss: 0.4658 - acc: 0.7709 - val_loss: 0.5441 - val_acc: 0.7199

Epoch 00084: val_loss did not improve from 0.52342
Epoch 85/500
37/37 [==============================] - 2s 44ms/step - loss: 0.4627 - acc:

In [153]:
from pathlib import Path
paths = sorted(Path(MODEL_SAVE_FOLDER_PATH).iterdir(), key=os.path.getmtime)[-1]
best_model_path = str(paths)
best_model_path

'model/rf_top10_seed42-41-0.5234.hdf5'

In [154]:
best_model = keras.models.load_model(best_model_path)
result = 0
for seed in range(0, 50):
    random.seed(seed)

    X = np.load(path + 'x_(7727,10,4068).npy')[:,:,rf_feature_index]
    y = np.load(path + 'y_(7727,1).npy')
    
    idx = list(range(len(x)))
    random.shuffle(idx)
    i = round(X.shape[0]*0.8)
    X_train, y_train = X[idx[:i],:,:], y[idx[:i]]
    X_test, y_test = X[idx[i:],:,:], y[idx[i:]]

    pred = best_model.predict(X_test)
    pred[pred>0.5]=1
    pred[pred<=0.5]=0
    acc = metrics.accuracy_score(y_test, pred)
    result += acc
    print(f'정확도 :{acc}, seed_num = {seed}')
print(f'평균 정확도 : {result/50}')

정확도 :0.7294498381877023, seed_num = 0
정확도 :0.7385113268608414, seed_num = 1
정확도 :0.7449838187702266, seed_num = 2
정확도 :0.7385113268608414, seed_num = 3
정확도 :0.7475728155339806, seed_num = 4
정확도 :0.7527508090614887, seed_num = 5
정확도 :0.7495145631067961, seed_num = 6
정확도 :0.7378640776699029, seed_num = 7
정확도 :0.7624595469255664, seed_num = 8
정확도 :0.7385113268608414, seed_num = 9
정확도 :0.7443365695792881, seed_num = 10
정확도 :0.7572815533980582, seed_num = 11
정확도 :0.7488673139158576, seed_num = 12
정확도 :0.7411003236245954, seed_num = 13
정확도 :0.7475728155339806, seed_num = 14
정확도 :0.7495145631067961, seed_num = 15
정확도 :0.7592233009708738, seed_num = 16
정확도 :0.7443365695792881, seed_num = 17
정확도 :0.7488673139158576, seed_num = 18
정확도 :0.7417475728155339, seed_num = 19
정확도 :0.7333333333333333, seed_num = 20
정확도 :0.7352750809061489, seed_num = 21
정확도 :0.7346278317152104, seed_num = 22
정확도 :0.7391585760517799, seed_num = 23
정확도 :0.7288025889967638, seed_num = 24
정확도 :0.7404530744336569, seed_num =

# With LSTM FI Top10

## RF

In [163]:
path = '/project/LSH/'
X =  np.load(path + 'x_(7727,4068).npy')[:,lstm_feature_index]
y = np.load(path + 'y_(7727,1).npy')

idx = list(range(len(X)))
random.shuffle(idx)

i = round(X.shape[0]*0.8)
X_train, y_train = X[idx[:i],:], y[idx[:i]]
X_test, y_test = X[idx[i:],:], y[idx[i:]]

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((6182, 30), (6182,), (1545, 30), (1545,))

In [164]:
model = RandomForestClassifier()
model.fit(X_train, y_train)

pred = model.predict(X_test)

print(f'정확도 : {metrics.accuracy_score(y_test, pred)}')

정확도 : 0.6608414239482201


## LSTM

In [165]:
import random    
seed_num = 42
random.seed(seed_num)
path = '/project/LSH/'
X = np.load(path + 'x_(7727,10,4068).npy')[:,:,lstm_feature_index]
y = np.load(path + 'y_(7727,1).npy')

idx = list(range(len(X)))
random.shuffle(idx)


i = round(X.shape[0]*0.8)
X_train, y_train = X[idx[:i],:,:], y[idx[:i]]
X_test, y_test = X[idx[i:],:,:], y[idx[i:]]

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((6182, 10, 30), (6182,), (1545, 10, 30), (1545,))

In [166]:
# ---------------------
seed_num = 42 
# ---------------------
tf.random.set_seed(seed_num)

lstm = Sequential()
lstm.add(InputLayer(input_shape=(X_train.shape[1],X_train.shape[2])))
lstm.add(LSTM(units=128, activation='hard_sigmoid', return_sequences=True))
lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
lstm.add(Dropout(0.2))
lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
lstm.add(LSTM(units=32, activation='hard_sigmoid', return_sequences=False))
lstm.add(Dropout(0.2))
lstm.add(Dense(units=1, activation='sigmoid'))

from tensorflow.keras.callbacks import ModelCheckpoint
import os

MODEL_SAVE_FOLDER_PATH = './model/'
if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
    os.mkdir(MODEL_SAVE_FOLDER_PATH)

model_path = MODEL_SAVE_FOLDER_PATH + 'lstm_top10_seed42-{epoch:02d}-{val_loss:.4f}.hdf5'

cb_checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss',
                                verbose=1, save_best_only=True, overwrite=True,)

early_stop = EarlyStopping(monitor='val_acc', patience=50, verbose=1, restore_best_weights=True)
lstm.compile(optimizer= keras.optimizers.Adam(learning_rate = 0.001), loss = "binary_crossentropy", metrics=['acc'])
lstm.fit(X_train, y_train, validation_split=0.25, batch_size=128, epochs=500,  callbacks=[early_stop,cb_checkpoint], shuffle=False)

Epoch 1/500
37/37 [==============================] - 6s 54ms/step - loss: 0.6759 - acc: 0.5984 - val_loss: 0.6660 - val_acc: 0.6177

Epoch 00001: val_loss improved from inf to 0.66603, saving model to ./model/lstm_top10_seed42-01-0.6660.hdf5
Epoch 2/500
37/37 [==============================] - 2s 44ms/step - loss: 0.6732 - acc: 0.6066 - val_loss: 0.6662 - val_acc: 0.6177

Epoch 00002: val_loss did not improve from 0.66603
Epoch 3/500
37/37 [==============================] - 2s 44ms/step - loss: 0.6740 - acc: 0.6085 - val_loss: 0.6655 - val_acc: 0.6177

Epoch 00003: val_loss improved from 0.66603 to 0.66551, saving model to ./model/lstm_top10_seed42-03-0.6655.hdf5
Epoch 4/500
37/37 [==============================] - 2s 44ms/step - loss: 0.6711 - acc: 0.6100 - val_loss: 0.6653 - val_acc: 0.6177

Epoch 00004: val_loss improved from 0.66551 to 0.66526, saving model to ./model/lstm_top10_seed42-04-0.6653.hdf5
Epoch 5/500
37/37 [==============================] - 2s 44ms/step - loss: 0.6706 -


Epoch 00037: val_loss improved from 0.59763 to 0.59715, saving model to ./model/lstm_top10_seed42-37-0.5972.hdf5
Epoch 38/500
37/37 [==============================] - 2s 44ms/step - loss: 0.5673 - acc: 0.6771 - val_loss: 0.5972 - val_acc: 0.6604

Epoch 00038: val_loss did not improve from 0.59715
Epoch 39/500
37/37 [==============================] - 2s 43ms/step - loss: 0.5664 - acc: 0.6695 - val_loss: 0.5968 - val_acc: 0.6598

Epoch 00039: val_loss improved from 0.59715 to 0.59684, saving model to ./model/lstm_top10_seed42-39-0.5968.hdf5
Epoch 40/500
37/37 [==============================] - 2s 44ms/step - loss: 0.5658 - acc: 0.6739 - val_loss: 0.5966 - val_acc: 0.6591

Epoch 00040: val_loss improved from 0.59684 to 0.59661, saving model to ./model/lstm_top10_seed42-40-0.5966.hdf5
Epoch 41/500
37/37 [==============================] - 2s 43ms/step - loss: 0.5668 - acc: 0.6689 - val_loss: 0.5958 - val_acc: 0.6611

Epoch 00041: val_loss improved from 0.59661 to 0.59576, saving model to .

37/37 [==============================] - 2s 44ms/step - loss: 0.5626 - acc: 0.6730 - val_loss: 0.5981 - val_acc: 0.6624

Epoch 00078: val_loss did not improve from 0.59266
Epoch 79/500
37/37 [==============================] - 2s 44ms/step - loss: 0.5608 - acc: 0.6721 - val_loss: 0.5978 - val_acc: 0.6624

Epoch 00079: val_loss did not improve from 0.59266
Epoch 80/500
37/37 [==============================] - 2s 44ms/step - loss: 0.5628 - acc: 0.6717 - val_loss: 0.6002 - val_acc: 0.6636

Epoch 00080: val_loss did not improve from 0.59266
Epoch 81/500
37/37 [==============================] - 2s 44ms/step - loss: 0.5633 - acc: 0.6713 - val_loss: 0.5954 - val_acc: 0.6643

Epoch 00081: val_loss did not improve from 0.59266
Epoch 82/500
37/37 [==============================] - 2s 44ms/step - loss: 0.5611 - acc: 0.6745 - val_loss: 0.5985 - val_acc: 0.6630

Epoch 00082: val_loss did not improve from 0.59266
Epoch 83/500
37/37 [==============================] - 2s 44ms/step - loss: 0.5594 - acc:

In [167]:
from pathlib import Path
paths = sorted(Path(MODEL_SAVE_FOLDER_PATH).iterdir(), key=os.path.getmtime)[-1]
best_model_path = str(paths)
best_model_path

'model/lstm_top10_seed42-59-0.5927.hdf5'

In [138]:
best_model = keras.models.load_model('./model/lstm_top10_seed42-84-0.6057.hdf5')
result = 0
for seed in range(0, 50):
    random.seed(seed)
    X = np.load(path + 'x_(7727,10,4068).npy')[:,:,lstm_feature_index]
    y = np.load(path + 'y_(7727,1).npy')
    
    idx = list(range(len(x)))
    random.shuffle(idx)
    i = round(X.shape[0]*0.8)
    X_train, y_train = X[idx[:i],:,:], y[idx[:i]]
    X_test, y_test = X[idx[i:],:,:], y[idx[i:]]

    pred = best_model.predict(X_test)
    pred[pred>0.5]=1
    pred[pred<=0.5]=0
    acc = metrics.accuracy_score(y_test, pred)
    result += acc
    print(f'정확도 :{acc}, seed_num = {seed}')
print(f'평균 정확도 : {result/50}')

정확도 :0.6550161812297735, seed_num = 0
정확도 :0.6511326860841424, seed_num = 1
정확도 :0.655663430420712, seed_num = 2
정확도 :0.6427184466019418, seed_num = 3
정확도 :0.6608414239482201, seed_num = 4
정확도 :0.6595469255663431, seed_num = 5
정확도 :0.6595469255663431, seed_num = 6
정확도 :0.6362459546925566, seed_num = 7
정확도 :0.6647249190938511, seed_num = 8
정확도 :0.654368932038835, seed_num = 9
정확도 :0.6362459546925566, seed_num = 10
정확도 :0.6686084142394823, seed_num = 11
정확도 :0.6498381877022654, seed_num = 12
정확도 :0.6491909385113268, seed_num = 13
정확도 :0.6498381877022654, seed_num = 14
정확도 :0.6653721682847896, seed_num = 15
정확도 :0.6750809061488673, seed_num = 16
정확도 :0.6498381877022654, seed_num = 17
정확도 :0.6563106796116505, seed_num = 18
정확도 :0.6414239482200648, seed_num = 19
정확도 :0.6660194174757281, seed_num = 20
정확도 :0.6608414239482201, seed_num = 21
정확도 :0.6427184466019418, seed_num = 22
정확도 :0.656957928802589, seed_num = 23
정확도 :0.6537216828478964, seed_num = 24
정확도 :0.6588996763754046, seed_num = 25